Инструкция: [https://github.com/zloishavrin/Easy-Wav2Lip](https://github.com/zloishavrin/Easy-Wav2Lip)

In [ ]:
version = 'v8.3'
#@title <h1>Шаг 1: Установка</h1> Достаточно просто нажать кнопку
#@markdown 👈 Нажмите на кнопку запуска. <br>
#@markdown > Рекомендуется предоставить доступ к Google Drive.
#@markdown <br> Или вы можете загрузить файлы вручную, но это будет дольше

#check if already installed
import os
import sys
if os.path.exists('installed.txt'):
  with open('last_file.txt', 'r') as file:
    last_file = file.readline()
  if last_file == version:
    sys.exit('Easy-Wav2Lip '+version+' запущен!')


#check GPU is enabled
print('checking for GPU')
import torch
if not torch.cuda.is_available():
  sys.exit('Нет доступа к GPU.')

#prompt to mount google drive
print('Запрос доступа к Google Drive')
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("Google Drive не подключен")


#start timer
import time
start_time = time.time()

#clone git
giturl = 'https://github.com/zloishavrin/Easy-Wav2Lip'


!git clone -b {version} {giturl}
%cd 'Easy-Wav2Lip'
working_directory = os.getcwd()
!mkdir 'face_alignment' 'temp'

#install prerequisites
print('installing batch_face')
import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        module='torchvision.transforms.functional_tensor')
!pip install batch_face --quiet
!pip install basicsr==1.4.2 --quiet
print('fixing basicsr degradations.py')
!cp /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py
print('installing gfpgan')
!pip install gfpgan --quiet

!python install.py

from IPython.display import clear_output
clear_output()
print("Загрузка выполнена. Перейдите ко второму шагу.")

#end timer
elapsed_time = time.time() - start_time
from easy_functions import format_time
print(f"Затрачено времени: {format_time(elapsed_time)}")

In [ ]:
if not os.path.exists('installed.txt'):
  sys.exit('Не удалось установить Easy-Wav2Lip')
time
############################## входные данные #####################################
#@markdown <h1>Шаг 2: Выберите входные данные:</h1>

# @markdown ПК: <h1></h1>Нажмите на значок папки ( 📁 ), найдите свой файл, щелкните правой кнопкой мыши, скопируйте путь, вставьте его ниже:
#@markdown<br></br>
# @markdown Мобильное устройство: <h1></h1>Нажмите на кнопку гамбургера ( ☰ ) в левом верхнем углу, нажмите показать файлы, найдите свой файл, зажмите его, скопируйте путь, вставьте ниже:
video_file = "" #@param {type:"string"}
vocal_file = "" #@param {type:"string"}

#@markdown > Оставьте vocal_file пустым, если в вашем видео уже закодирован нужный речевой звук.
#@markdown # Качество
quality = "Enhanced" # @param ["Fast", "Improved", "Enhanced"]
#@markdown * <b><u>Fast</u></b>: Wav2Lip <br>
#@markdown * <b><u>Improved</u></b>: Wav2Lip с раширенной маской вокруг рта <br>
#@markdown * <b><u>Enhanced</u></b>: Wav2Lip + mask + GFPGAN увеличение разрешения лица
output_height = "full resolution" #@param ["half resolution", "full resolution", "480"] {allow-input: true}
use_previous_tracking_data = True #@param {type:"boolean"}
#@markdown Ускоряет обработку одного и того же видео, используемого несколько раз - он должен автоматически удалять последний файл отслеживания при смене видео, но если он не работает после первого видео, снимите этот флажок.

#@markdown
#------------------------------*Шаг 3*----------------------------------------!
#@markdown <h1>👈 Шаг 3: Нажмите на кнопку воспроизведения в этой ячейке. </h1> (ctrl + F10) - Затем дождитесь завершения обработки.
# масштабировать подложку в зависимости от разрешения
#@markdown <br>

#@markdown ---
#@markdown <br>

#@markdown # [Расширенная настройка](https://github.com/zloishavrin/Easy-Wav2Lip) (опционально) </h1>Если вы новичок, просто проигнорируйте все это или ознакомьтесь с инструкцией.
wav2lip_version = "Wav2Lip" # @param ["Wav2Lip", "Wav2Lip_GAN"]
nosmooth = True #@param {type:"boolean"}
#@markdown ### Внутренние отступы маски:<h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-100, max:100, step:1}
D = 10 #@param {type:"slider", min:-100, max:100, step:1}
L = 0 #@param {type:"slider", min:-100, max:100, step:1}
R = 0 #@param {type:"slider", min:-100, max:100, step:1}
#@markdown <br>

#@markdown ### Настройка маски
size = 1.5 #@param {type:"slider", min:1, max:6, step:0.1}
feathering = 1 #@param {type:"slider", min:0, max:3, step:1}
mouth_tracking = False #@param {type:"boolean"}
debug_mask = False #@param {type:"boolean"}


#@markdown # Другие настройки

batch_process = False #@param {type:"boolean"}
output_suffix = "_Easy-Wav2Lip" #@param {type:"string"}
include_settings_in_suffix = False #@param {type:"boolean"}
preview_input = False #@param {type:"boolean"}
preview_settings = False #@param {type:"boolean"}
#@markdown preview_settings обрабатывает только один кадр, чтобы вы могли увидеть, как это выглядит, не снимая видео целиком
frame_to_preview = 100 # @param {type:"integer"}


import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Put all your variables in a dictionary
options = {
    'video_file': video_file,
    'vocal_file': vocal_file,
    'quality': quality,
    'output_height': output_height,
    'wav2lip_version': wav2lip_version,
    'use_previous_tracking_data': use_previous_tracking_data,
    'nosmooth': nosmooth
}
padding = {
    'U': U,
    'D': D,
    'L': L,
    'R': R
}
mask = {
    'size': size,
    'feathering': feathering,
    'mouth_tracking': mouth_tracking,
    'debug_mask': debug_mask
}
other = {
    'batch_process': batch_process,
    'output_suffix': output_suffix,
    'include_settings_in_suffix': include_settings_in_suffix,
    'preview_input': preview_input,
    'preview_settings': preview_settings,
    'frame_to_preview': frame_to_preview
}


# Add the dictionary to the ConfigParser object
config['OPTIONS'] = options
config['PADDING'] = padding
config['MASK'] = mask
config['OTHER'] = other

# Write the data to an INI file
with open('config.ini', 'w') as f:
    config.write(f)

!python run.py

from easy_functions import show_video
from IPython.display import Image
if preview_settings:
  if os.path.isfile(os.path.join('temp','preview.jpg')):
    display(Image(os.path.join('temp','preview.jpg')))
else:
  if os.path.isfile(os.path.join('temp','output.mp4')):
    print(f"Loading video preview...")
    show_video(os.path.join('temp','output.mp4'))